In [ ]:
!pip install torchinfo

In [ ]:
import time
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
import numpy as np
import cv2
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms
from torchinfo import summary
import torchvision.models as models
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
import PIL
import matplotlib.pyplot as plt
import seaborn as sns
import time
from collections import OrderedDict
import platform
import psutil
import random
import glob
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from torchvision.transforms import ColorJitter, RandomRotation, RandomResizedCrop
from torchvision.transforms.functional import gaussian_blur
from PIL import ImageOps
from tabulate import tabulate
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, confusion_matrix, roc_auc_score, roc_curve, auc)
import matplotlib.pyplot as plt

In [ ]:
cpu_info = platform.processor()

ram_info = psutil.virtual_memory()
total_ram_gb = ram_info.total / (1024 ** 3)

try:
    gpu_info = !nvidia-smi --query-gpu=gpu_name --format=csv
    gpu_name = gpu_info[1]
except:
    gpu_name = "No GPU available"

print("CPU:", cpu_info)
print("Total RAM (GB):", round(total_ram_gb, 2))
print("GPU:", gpu_name)

In [ ]:
seed = 1
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [ ]:
data_dir = "/kaggle/input/resized-fish"

file_paths = []
labels = []

for class_name in os.listdir(data_dir):
    class_dir = os.path.join(data_dir, class_name)
    for image_name in os.listdir(class_dir):
        file_paths.append(os.path.join(class_dir, image_name))
        labels.append(class_name)

df = pd.DataFrame({"file_path": file_paths, "label": labels})
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
class_counts_train = df['label'].value_counts()

for class_name, count in class_counts_train.items():
    print(f"Class: {class_name}, Count: {count}")

plt.figure(figsize=(15, 4))
ax = class_counts_train.plot(kind='bar')
plt.xlabel('Classes')
plt.ylabel('Amount of data')
plt.xticks(rotation=360)
for i, count in enumerate(class_counts_train):
    ax.text(i, count + 5, str(count), ha='center')
plt.ylim(0, max(class_counts_train) * 1.2)
plt.show()

In [ ]:
random_index = random.randint(1, len(df) - 1)
random_row = df.iloc[random_index]

file_path = random_row['file_path']
label = random_row['label']

image = Image.open(file_path)
plt.imshow(image)
plt.title(f"Label: {label}")
plt.axis('off')
plt.show()

In [ ]:
train_dataframe, temp_dataframe = train_test_split(df, test_size=0.30, 
                                     stratify=df['label'], random_state=42)
valid_dataframe, test_df = train_test_split(temp_dataframe, test_size=0.50, 
                                            stratify=temp_dataframe['label'], random_state=42)

In [ ]:
print("Training Data: ", len(train_dataframe))
print("Validation Data: ", len(valid_dataframe))
print("Test Data: ", len(test_df))
print("-------------------------------------------")
print("Total amounts of data in the dataset: ", len(df))

In [ ]:
class_counts_train = train_dataframe['label'].value_counts()
class_counts_valid = valid_dataframe['label'].value_counts()
class_counts_test = test_df['label'].value_counts()

train_table_data = [[class_name, count] for class_name, count in class_counts_train.items()]
valid_table_data = [[class_name, count] for class_name, count in class_counts_valid.items()]
test_table_data = [[class_name, count] for class_name, count in class_counts_test.items()]

print("Train Dataset")
print(tabulate(train_table_data, headers=["Class", "Count"]))
print("\nValidation Dataset")
print(tabulate(valid_table_data, headers=["Class", "Count"]))
print("\nTest Dataset")
print(tabulate(test_table_data, headers=["Class", "Count"]))

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, dataframe, normalize, is_lb=False):
        self.dataframe = dataframe
        self.normalize = normalize
        self.is_lb = is_lb
        class_names = sorted(self.dataframe['label'].unique())  # Ensure consistent order
        self.label_map = {class_name: idx for idx, class_name in enumerate(class_names)}

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        img_path = self.dataframe.iloc[index]['file_path']
        image = PIL.Image.open(img_path)

        if self.is_lb:
            label = self.label_map[self.dataframe.iloc[index]['label']]
            return self.normalize(image), label
        else:
            return self.normalize(image), self.normalize(image)

In [ ]:
normalize = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_dataset = ImageDataset(train_dataframe, normalize, is_lb=True)
print(train_dataset.label_map)

In [ ]:
save_path_checkpoints = "/kaggle/working/"
os.makedirs(save_path_checkpoints, exist_ok=True)

In [ ]:
train_batch = 32
val_batch = 8

# Define normalization transform
normalize = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Create datasets (globally accessible)
train_dataset = ImageDataset(train_dataframe, normalize, is_lb=True)
valid_dataset = ImageDataset(valid_dataframe, normalize, is_lb=True)
test_dataset = ImageDataset(test_df, normalize, is_lb=True)

# Create data loaders
dataloader_train_dataset = DataLoader(train_dataset, batch_size=train_batch, shuffle=True, num_workers=2)
dataloader_valid_dataset = DataLoader(valid_dataset, batch_size=val_batch, shuffle=False, num_workers=2)
dataloader_test_dataset = DataLoader(test_dataset, batch_size=val_batch, shuffle=False, num_workers=2)

# Attention blocks

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Squeeze-and-Excitation Block
class SEBlock(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

# Channel Attention Module (part of CBAM)
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        
        self.fc1 = nn.Conv2d(in_planes, in_planes // ratio, 1, bias=False)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Conv2d(in_planes // ratio, in_planes, 1, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = self.fc2(self.relu1(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu1(self.fc1(self.max_pool(x))))
        out = avg_out + max_out
        return self.sigmoid(out)

# Spatial Attention Module (part of CBAM)
class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv1 = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)

# CBAM (Convolutional Block Attention Module)
class CBAM(nn.Module):
    def __init__(self, in_planes, ratio=16, kernel_size=7):
        super(CBAM, self).__init__()
        self.ca = ChannelAttention(in_planes, ratio)
        self.sa = SpatialAttention(kernel_size)

    def forward(self, x):
        x = self.ca(x) * x
        x = self.sa(x) * x
        return x

# Self-Attention Module
class SelfAttention(nn.Module):
    def __init__(self, in_dim):
        super(SelfAttention, self).__init__()
        self.query_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim//8, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim//8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        m_batchsize, C, width, height = x.size()
        proj_query = self.query_conv(x).view(m_batchsize, -1, width*height).permute(0, 2, 1)
        proj_key = self.key_conv(x).view(m_batchsize, -1, width*height)
        energy = torch.bmm(proj_query, proj_key)
        attention = self.softmax(energy)
        proj_value = self.value_conv(x).view(m_batchsize, -1, width*height)
        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        out = out.view(m_batchsize, C, width, height)
        out = self.gamma * out + x
        return out

# GCNet Attention (Global Context Block)
class GCBlock(nn.Module):
    def __init__(self, in_channels, ratio=16):
        super(GCBlock, self).__init__()
        self.pool_h = nn.AdaptiveAvgPool2d((None, 1))
        self.pool_w = nn.AdaptiveAvgPool2d((1, None))
        
        mip = max(8, in_channels // ratio)
        
        self.conv1 = nn.Conv2d(in_channels, mip, kernel_size=1, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(mip)
        self.act = nn.ReLU(inplace=True)
        
        self.conv_h = nn.Conv2d(mip, in_channels, kernel_size=1, stride=1, padding=0)
        self.conv_w = nn.Conv2d(mip, in_channels, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        identity = x
        
        n, c, h, w = x.size()
        x_h = self.pool_h(x)
        x_w = self.pool_w(x).permute(0, 1, 3, 2)
        
        y = torch.cat([x_h, x_w], dim=2)
        y = self.conv1(y)
        y = self.bn1(y)
        y = self.act(y)
        
        x_h, x_w = torch.split(y, [h, w], dim=2)
        x_w = x_w.permute(0, 1, 3, 2)
        
        a_h = self.conv_h(x_h).sigmoid()
        a_w = self.conv_w(x_w).sigmoid()
        
        out = identity * a_w * a_h
        return out

In [ ]:
# Enhanced FishNet with Attention Mechanisms
class FishNetWithAttention(nn.Module):
    def __init__(self, num_classes, attention_type='cbam'):
        super(FishNetWithAttention, self).__init__()
        self.attention_type = attention_type
        
        # Feature extraction layers
        self.features = nn.Sequential()
        
        # First block
        self.features.add_module('conv1', nn.Conv2d(3, 32, 3, padding=1))
        self.features.add_module('bn1', nn.BatchNorm2d(32))
        self.features.add_module('relu1', nn.ReLU())
        self.features.add_module('pool1', nn.MaxPool2d(2, 2))
        
        # Add attention after first block
        if attention_type == 'se':
            self.features.add_module('se1', SEBlock(32))
        elif attention_type == 'cbam':
            self.features.add_module('cbam1', CBAM(32))
        elif attention_type == 'self':
            self.features.add_module('self1', SelfAttention(32))
        elif attention_type == 'gc':
            self.features.add_module('gc1', GCBlock(32))
        
        # Second block
        self.features.add_module('conv2', nn.Conv2d(32, 64, 3, padding=1))
        self.features.add_module('bn2', nn.BatchNorm2d(64))
        self.features.add_module('relu2', nn.ReLU())
        self.features.add_module('pool2', nn.MaxPool2d(2, 2))
        
        # Add attention after second block
        if attention_type == 'se':
            self.features.add_module('se2', SEBlock(64))
        elif attention_type == 'cbam':
            self.features.add_module('cbam2', CBAM(64))
        elif attention_type == 'self':
            self.features.add_module('self2', SelfAttention(64))
        elif attention_type == 'gc':
            self.features.add_module('gc2', GCBlock(64))
        
        # Third block
        self.features.add_module('conv3', nn.Conv2d(64, 128, 3, padding=1))
        self.features.add_module('bn3', nn.BatchNorm2d(128))
        self.features.add_module('relu3', nn.ReLU())
        self.features.add_module('pool3', nn.MaxPool2d(2, 2))
        
        # Add attention after third block
        if attention_type == 'se':
            self.features.add_module('se3', SEBlock(128))
        elif attention_type == 'cbam':
            self.features.add_module('cbam3', CBAM(128))
        elif attention_type == 'self':
            self.features.add_module('self3', SelfAttention(128))
        elif attention_type == 'gc':
            self.features.add_module('gc3', GCBlock(128))
        
        # Fourth block
        self.features.add_module('conv4', nn.Conv2d(128, 256, 3, padding=1))
        self.features.add_module('bn4', nn.BatchNorm2d(256))
        self.features.add_module('relu4', nn.ReLU())
        self.features.add_module('pool4', nn.MaxPool2d(2, 2))
        
        # Add attention after fourth block
        if attention_type == 'se':
            self.features.add_module('se4', SEBlock(256))
        elif attention_type == 'cbam':
            self.features.add_module('cbam4', CBAM(256))
        elif attention_type == 'self':
            self.features.add_module('self4', SelfAttention(256))
        elif attention_type == 'gc':
            self.features.add_module('gc4', GCBlock(256))
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 14 * 14, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [ ]:
# Training function
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=100, save_path_checkpoints="checkpoints"):
    train_loss_history = []
    val_loss_history = []
    train_acc_history = []
    val_acc_history = []

    best_val_acc = 0.0

    # Make sure checkpoint directory exists
    import os
    os.makedirs(save_path_checkpoints, exist_ok=True)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        epoch_train_loss = running_loss / total
        epoch_train_acc = correct / total

        train_loss_history.append(epoch_train_loss)
        train_acc_history.append(epoch_train_acc)

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for val_images, val_labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
                val_images = val_images.to(device)
                val_labels = val_labels.to(device)

                val_outputs = model(val_images)
                loss = criterion(val_outputs, val_labels)

                val_loss += loss.item() * val_images.size(0)
                _, val_preds = torch.max(val_outputs, 1)
                val_correct += (val_preds == val_labels).sum().item()
                val_total += val_labels.size(0)

        epoch_val_loss = val_loss / val_total
        epoch_val_acc = val_correct / val_total

        val_loss_history.append(epoch_val_loss)
        val_acc_history.append(epoch_val_acc)

        # Save checkpoint if improved
        if epoch_val_acc > best_val_acc:
            best_val_acc = epoch_val_acc
            torch.save({
                'epoch': epoch,
                'model_weight': model.state_dict(),
                'optimizer_state': optimizer.state_dict()
            }, f"{save_path_checkpoints}/model.pt")

        print(f"Epoch {epoch+1}/{num_epochs} | "
              f"Train Loss: {epoch_train_loss:.4f} | Train Acc: {epoch_train_acc:.4f} | "
              f"Val Loss: {epoch_val_loss:.4f} | Val Acc: {epoch_val_acc:.4f}")

    print(f"Best Validation Accuracy: {best_val_acc:.4f}")

    return train_loss_history, train_acc_history, val_loss_history, val_acc_history

In [ ]:
# Evaluation function
def evaluate_model(model, criterion, dataloader):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    loss = running_loss / total
    accuracy = correct / total

    return loss, accuracy

# Class accuracy function
def class_accuracy(model, dataloader, num_classes):
    class_correct = [0.0] * num_classes
    class_total = [0.0] * num_classes

    model.eval()
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct = (predicted == labels)
            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += correct[i].item()
                class_total[label] += 1

    class_accuracy = [class_correct[i] / class_total[i] if class_total[i] > 0 else 0 for i in range(num_classes)]

    return class_accuracy

# Feature extraction function
def extract_features(extractor, data_loader, device):
    extractor.eval()
    features_list = []
    labels_list = []
    with torch.no_grad():
        for images, labels in tqdm(data_loader, desc="Extracting Features"):
            images = images.to(device)
            feats = extractor(images)  # Shape: [batch, 256]
            features_list.append(feats.cpu().numpy())
            labels_list.append(labels.numpy())
    return np.concatenate(features_list, axis=0), np.concatenate(labels_list, axis=0)

# Function to create different attention-based models
def create_attention_model(num_classes, attention_type):
    return FishNetWithAttention(num_classes, attention_type).to(device)

# Feature extractor for attention models
def create_feature_extractor_with_attention(original_model):
    class FishNetFeaturesWithAttention(nn.Module):
        def __init__(self, original_model):
            super(FishNetFeaturesWithAttention, self).__init__()
            # Extract feature layers
            self.features = original_model.features
            # Extract classifier layers up to 256-D features
            self.classifier_part = nn.Sequential(*list(original_model.classifier.children())[:-3])
        
        def forward(self, x):
            x = self.features(x)
            x = self.classifier_part(x)
            return x
    
    return FishNetFeaturesWithAttention(original_model).to(device)

In [ ]:
# Now let's run the attention-based models
attention_types = ['se']
attention_results = {}

print("Starting Attention-Based Model Training...")

# Train and evaluate each attention-based model
for attention_type in attention_types:
    print(f"\n{'='*50}")
    print(f"Training FishNet with {attention_type.upper()} Attention")
    print(f"{'='*50}")
    
    # Create model
    model_attn = create_attention_model(num_classes=20, attention_type=attention_type)
    
    # Define loss and optimizer
    criterion_attn = nn.CrossEntropyLoss()
    optimizer_attn = torch.optim.Adam(model_attn.parameters(), lr=0.0001)
    
    # Train the model
    start_time = time.time()
    train_loss_history_attn, train_acc_history_attn, val_loss_history_attn, val_acc_history_attn = train_model(
        model_attn, criterion_attn, optimizer_attn,
        dataloader_train_dataset, dataloader_valid_dataset,
        num_epochs=100,
        save_path_checkpoints=f"checkpoints_{attention_type}"
    )
    training_time_attn = time.time() - start_time
    
    # Save training history
    data_attn = {
        'Epoch': list(range(1, len(train_loss_history_attn) + 1)),
        'Train Loss': train_loss_history_attn,
        'Train Accuracy': train_acc_history_attn,
        'Validation Loss': val_loss_history_attn,
        'Validation Accuracy': val_acc_history_attn
    }
    history_attn = pd.DataFrame(data_attn)
    history_attn.to_excel(f'/kaggle/working/training_data_{attention_type}.xlsx', index=False)
    
    # Load best model
    checkpoint_path_attn = f"checkpoints_{attention_type}/model.pt"
    checkpoint_attn = torch.load(checkpoint_path_attn, map_location=device)
    model_attn.load_state_dict(checkpoint_attn['model_weight'])
    model_attn.eval()
    
    # Evaluate on test set
    start_time = time.time()
    test_loss_attn, test_accuracy_attn = evaluate_model(model_attn, criterion_attn, dataloader_test_dataset)
    test_time_attn = time.time() - start_time
    
    print(f"Test Accuracy for {attention_type.upper()}: {test_accuracy_attn:.4f}")
    print(f"Training Time: {training_time_attn/60:.2f} minutes")
    print(f"Test Time: {test_time_attn:.2f} seconds")
    
    # Store results
    attention_results[attention_type] = {
        'model': model_attn,
        'test_accuracy': test_accuracy_attn,
        'training_time': training_time_attn,
        'test_time': test_time_attn,
        'checkpoint_path': checkpoint_path_attn
    }

# Extract features and apply ML classifiers for each attention model
ml_results_attention = {}

for attention_type in attention_types:
    print(f"\nProcessing {attention_type.upper()} Attention Model for ML Classifiers...")
    
    # Load best model
    model_attn = attention_results[attention_type]['model']
    
    # Create feature extractor
    feature_extractor_attn = create_feature_extractor_with_attention(model_attn)
    feature_extractor_attn.eval()
    
    # Extract features
    X_train_feats_attn, y_train_attn = extract_features(feature_extractor_attn, dataloader_train_dataset, device)
    X_test_feats_attn, y_test_attn = extract_features(feature_extractor_attn, dataloader_test_dataset, device)
    
    # Scale features
    scaler_attn = StandardScaler()
    X_train_scaled_attn = scaler_attn.fit_transform(X_train_feats_attn)
    X_test_scaled_attn = scaler_attn.transform(X_test_feats_attn)
    
    # Train ML classifiers
    classifiers_attn = {
        "SVM": SVC(kernel='rbf', probability=True, random_state=42),
        "KNN": KNeighborsClassifier(n_neighbors=5),
        "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1),
        "XGBoost": XGBClassifier(n_estimators=100, eval_metric='mlogloss', random_state=42, n_jobs=-1)
    }
    
    results_attn = []
    y_pred_dict_attn = {}
    y_proba_dict_attn = {}
    
    for name, clf in classifiers_attn.items():
        print(f"  Training {name}...")
        start = time.time()
        
        # Fit
        if name == "SVM":
            clf.fit(X_train_feats_attn, y_train_attn)
            train_time = time.time() - start
            
            # Predict
            pred_start = time.time()
            y_pred = clf.predict(X_test_feats_attn)
            y_proba = clf.predict_proba(X_test_feats_attn)
            test_time = time.time() - pred_start
        else:
            clf.fit(X_train_scaled_attn, y_train_attn)
            train_time = time.time() - start
            
            # Predict
            pred_start = time.time()
            y_pred = clf.predict(X_test_scaled_attn)
            y_proba = clf.predict_proba(X_test_scaled_attn)
            test_time = time.time() - pred_start

        # Metrics
        acc = accuracy_score(y_test_attn, y_pred)
        prec = precision_score(y_test_attn, y_pred, average='weighted')
        rec = recall_score(y_test_attn, y_pred, average='weighted')
        f1 = f1_score(y_test_attn, y_pred, average='weighted')
        
        try:
            auc_score = roc_auc_score(y_test_attn, y_proba, average='weighted', multi_class='ovr') if y_proba.shape[1] > 2 else roc_auc_score(y_test_attn, y_proba[:, 1])
        except:
            auc_score = float('nan')

        # Store results
        results_attn.append({
            "Model": f"{attention_type.upper()}+{name}",
            "Accuracy": acc,
            "Precision": prec,
            "Recall": rec,
            "F1-Score": f1,
            "ROC-AUC": auc_score,
            "Train Time (s)": train_time,
            "Test Time (s)": test_time
        })
        y_pred_dict_attn[name] = y_pred
        y_proba_dict_attn[name] = y_proba
    
    # Store results for this attention type
    ml_results_attention[attention_type] = pd.DataFrame(results_attn)

print("Attention-based model training and evaluation completed!")

# Create comprehensive comparison results
print("\n" + "="*80)
print("="*80)

# Combine all results
all_results = []

# Add attention-based results
for attention_type in attention_types:
    # Add attention-only CNN results
    attn_cnn_result = {
        "Model": f"{attention_type.upper()} CNN",
        "Accuracy": attention_results[attention_type]['test_accuracy'],
        "Precision": "N/A",
        "Recall": "N/A",
        "F1-Score": "N/A",
        "ROC-AUC": "N/A",
        "Train Time (s)": attention_results[attention_type]['training_time'],
        "Test Time (s)": attention_results[attention_type]['test_time']
    }
    all_results.append(attn_cnn_result)
    
    # Add hybrid attention+ML results
    for _, row in ml_results_attention[attention_type].iterrows():
        all_results.append(row.to_dict())

In [ ]:

# Create final comparison DataFrame
comparison_df = pd.DataFrame(all_results)
print(comparison_df.round(4))

# Save comparison results
comparison_df.to_excel('/kaggle/working/comparison_results.xlsx', index=False)

# Detailed evaluation for the best attention model
best_attention_type = max(attention_results.keys(), key=lambda k: attention_results[k]['test_accuracy'])
print(f"\nDetailed Evaluation for Best Attention Model: {best_attention_type.upper()}")

# Load best attention model
best_model = attention_results[best_attention_type]['model']

# Class accuracy for best attention model
class_acc_attn = class_accuracy(best_model, dataloader_test_dataset, 20)
for i in range(20):
    print(f"Class {i} Accuracy: {class_acc_attn[i]:.4f}")

print(f"\nAttention-based models implementation completed successfully!")
print(f"All results saved to /kaggle/working/comparison_results.xlsx")
# Classification report for best attention model
y_true_attn = []
y_pred_attn = []

best_model.eval()
with torch.no_grad():
    for inputs, labels in dataloader_test_dataset:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = best_model(inputs)
        _, predicted = torch.max(outputs, 1)
        y_true_attn.extend(labels.cpu().numpy())
        y_pred_attn.extend(predicted.cpu().numpy())

# Confusion Matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_true_attn, y_pred_attn)
plt.figure(figsize=(15, 12))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title(f'Confusion Matrix - {best_attention_type.upper()} Attention Model')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.savefig(f'/kaggle/working/confusion_matrix_{best_attention_type}.png')
plt.show()

# Classification Report
class_names = [f"Class_{i}" for i in range(20)]
print(f"\nClassification Report for {best_attention_type.upper()} Attention Model:")
print(classification_report(y_true_attn, y_pred_attn, target_names=class_names))

# ROC Curve (for multi-class)
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from itertools import cycle

# Binarize the output for ROC curve
y_test_binarized = label_binarize(y_true_attn, classes=range(20))
n_classes = y_test_binarized.shape[1]

# Get probabilities from the best attention model
y_proba_attn = []
best_model.eval()
with torch.no_grad():
    for inputs, _ in dataloader_test_dataset:
        inputs = inputs.to(device)
        outputs = best_model(inputs)
        probs = torch.softmax(outputs, dim=1)
        y_proba_attn.extend(probs.cpu().numpy())

y_proba_attn = np.array(y_proba_attn)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_binarized[:, i], y_proba_attn[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test_binarized.ravel(), y_proba_attn.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Plot ROC curves
plt.figure(figsize=(15, 10))
colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'red', 'green', 'purple', 'brown', 'pink', 'gray', 'olive',
                'cyan', 'magenta', 'yellow', 'black', 'lime', 'teal', 'lavender', 'maroon', 'navy', 'gold'])

for i, color in zip(range(min(10, n_classes)), colors):  # Plot first 10 classes for clarity
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label=f'ROC curve of class {i} (area = {roc_auc[i]:.2f})')

plt.plot(fpr["micro"], tpr["micro"],
         label=f'Micro-average ROC curve (area = {roc_auc["micro"]:.2f})',
         color='deeppink', linestyle=':', linewidth=4)

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(f'ROC Curves - {best_attention_type.upper()} Attention Model')
plt.legend(loc="lower right")
plt.savefig(f'/kaggle/working/roc_curves_{best_attention_type}.png')
plt.show()

# Summary of improvements
print("\n" + "="*80)
print("SUMMARY OF IMPROVEMENTS")
print("="*80)

baseline_acc = 0.980
best_attention_acc = attention_results[best_attention_type]['test_accuracy']

print(f"Baseline CNN Accuracy: {baseline_acc:.4f}")
print(f"({best_attention_type.upper()}) CNN Accuracy: {best_attention_acc:.4f}")
print(f"Improvement: {best_attention_acc - baseline_acc:.4f} ({((best_attention_acc - baseline_acc)/baseline_acc)*100:.2f}%)")

# Create final comparison DataFrame
comparison_df = pd.DataFrame(all_results)
print("\nFinal Comparison Results:")
print(comparison_df.round(4))

# Save comparison results
comparison_df.to_excel('/kaggle/working/comparison_results.xlsx', index=False)

# Detailed evaluation for the best attention model
best_attention_type = max(attention_results.keys(), key=lambda k: attention_results[k]['test_accuracy'])
print(f"\nDetailed Evaluation for Best Attention Model: {best_attention_type.upper()}")

# Load best attention model
best_model = attention_results[best_attention_type]['model']

# Class accuracy for best attention model
class_acc_attn = class_accuracy(best_model, dataloader_test_dataset, 20)
print("\nClass-wise Accuracy for Best Attention Model:")
for i in range(20):
    print(f"Class {i} Accuracy: {class_acc_attn[i]:.4f}")

print(f"\nAttention-based models implementation completed successfully!")
print(f"All results saved to /kaggle/working/comparison_results.xlsx")
# Classification report for best attention model
y_true_attn = []
y_pred_attn = []

best_model.eval()
with torch.no_grad():
    for inputs, labels in dataloader_test_dataset:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = best_model(inputs)
        _, predicted = torch.max(outputs, 1)
        y_true_attn.extend(labels.cpu().numpy())
        y_pred_attn.extend(predicted.cpu().numpy())


In [ ]:
# Performance comparison table
performance_comparison = []
for attn_type in attention_types:
    performance_comparison.append({
        'Attention Type': attn_type.upper(),
        'Test Accuracy': attention_results[attn_type]['test_accuracy'],
        'Training Time (min)': attention_results[attn_type]['training_time']/60,
        'Test Time (sec)': attention_results[attn_type]['test_time']
    })

perf_df = pd.DataFrame(performance_comparison)
print(perf_df.round(4))

# Save performance comparison
perf_df.to_excel('/kaggle/working/attention_performance_comparison.xlsx', index=False)

print(f"\nAll results have been saved to Excel files:")
print(f"- /kaggle/working/comparison_results.xlsx")
print(f"- /kaggle/working/attention_performance_comparison.xlsx")
print(f"- /kaggle/working/confusion_matrix_{best_attention_type}.png")
print(f"- /kaggle/working/roc_curves_{best_attention_type}.png")

# Training curves for attention models
for attention_type in attention_types:
    try:
        history_attn = pd.read_excel(f'/kaggle/working/training_data_{attention_type}.xlsx')
        
        plt.figure(figsize=(15, 5))
        
        plt.subplot(1, 3, 1)
        plt.plot(history_attn['Epoch'], history_attn['Train Loss'], label='Train Loss')
        plt.plot(history_attn['Epoch'], history_attn['Validation Loss'], label='Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.title(f'{attention_type.upper()} Attention - Loss Curves')
        plt.legend()
        plt.grid(True)
        
        plt.subplot(1, 3, 2)
        plt.plot(history_attn['Epoch'], history_attn['Train Accuracy'], label='Train Accuracy')
        plt.plot(history_attn['Epoch'], history_attn['Validation Accuracy'], label='Validation Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.title(f'{attention_type.upper()} Attention - Accuracy Curves')
        plt.legend()
        plt.grid(True)
        
        plt.subplot(1, 3, 3)
        plt.plot(history_attn['Epoch'], history_attn['Validation Loss'], 'r-', label='Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Validation Loss')
        plt.title(f'{attention_type.upper()} Attention - Validation Loss')
        plt.legend()
        plt.grid(True)
        
        plt.tight_layout()
        plt.savefig(f'/kaggle/working/training_curves_{attention_type}.png')
        plt.show()
    except Exception as e:
        print(f"Could not plot training curves for {attention_type}: {e}")

print("\nTraining curves saved for all attention mechanisms.")

In [ ]:
# ==================== GRAD-CAM SECTION ADDED HERE ====================
# Define GradCAM class
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        target_layer.register_forward_hook(self.save_activation)
        target_layer.register_backward_hook(self.save_gradient)
    def save_activation(self, module, input, output):
        self.activations = output.detach()
    def save_gradient(self, module, grad_input, grad_output):
        self.gradients = grad_output[0].detach()
    def generate_cam(self, input_tensor, class_idx=None):
        output = self.model(input_tensor)
        if class_idx is None:
            class_idx = output.argmax(dim=1).item()
        self.model.zero_grad()
        target = output[:, class_idx]
        target.backward()
        weights = self.gradients.mean(dim=(2,3), keepdim=True)
        cam = (weights * self.activations).sum(dim=1, keepdim=True)
        cam = torch.relu(cam)
        cam = cam.squeeze().cpu().numpy()
        cam = cv2.resize(cam, (224,224))
        cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)
        return cam

# Apply Grad-CAM to the best attention model
print(f"\nGenerating Grad-CAM for {best_attention_type.upper()} Attention Model...")

# Select the last conv layer
target_layer = best_model.features.conv4
gradcam = GradCAM(model=best_model, target_layer=target_layer)

# Pick a test image
img_tensor, true_label = test_dataset[0]
img_tensor_with_batch = img_tensor.unsqueeze(0).to(device)

# Get prediction
with torch.no_grad():
    output = best_model(img_tensor_with_batch)
    pred_label = output.argmax(dim=1).item()

# Generate CAM
cam = gradcam.generate_cam(img_tensor_with_batch, class_idx=pred_label)

# Denormalize image
img = img_tensor.permute(1, 2, 0).cpu().numpy()
img = (img - img.min()) / (img.max() - img.min())

# Plot
plt.figure(figsize=(12, 6))
plt.subplot(1, 3, 1)
plt.imshow(img)
plt.title(f"Original Image\nTrue: {true_label}, Pred: {pred_label}")
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(cam, cmap='jet')
plt.title(f"Grad-CAM Heatmap\nClass: {pred_label}")
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(img)
plt.imshow(cam, cmap='jet', alpha=0.5)
plt.title("Grad-CAM Overlay")
plt.axis('off')

plt.tight_layout()
plt.savefig(f"/kaggle/working/gradcam_{best_attention_type}_overlay.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"Grad-CAM successfully applied and saved.")

In [ ]:
# ==================== INFERENCE TIME PER CLASS ====================

print(f"\nMeasuring inference time per class for {best_attention_type.upper()} Attention Model...")

# Set model to evaluation mode
best_model.eval()

# Lists to store results
class_names_list = list(train_dataset.label_map.keys())  # ['Class_0', ..., 'Class_19'] or actual species names
class_times = {cls: [] for cls in class_names_list}  # Accumulate times per class
label_to_name = {idx: name for name, idx in train_dataset.label_map.items()}

# Warm-up (important for accurate timing on GPU)
with torch.no_grad():
    dummy_input = torch.randn(1, 3, 224, 224).to(device)
    for _ in range(5):
        _ = best_model(dummy_input)

# Inference with timing
with torch.no_grad():
    for img_tensor, label in tqdm(test_dataset, desc="Timing Inference"):
        img_tensor_with_batch = img_tensor.unsqueeze(0).to(device)
        
        # Synchronize before timing (for accurate GPU timing)
        if device.type == 'cuda':
            torch.cuda.synchronize()
        start_time = time.time()
        
        # Forward pass
        output = best_model(img_tensor_with_batch)
        
        # Synchronize after (for accurate GPU timing)
        if device.type == 'cuda':
            torch.cuda.synchronize()
        end_time = time.time()
        
        inference_time = end_time - start_time
        class_name = label_to_name[label]
        class_times[class_name].append(inference_time)

# Compute average time per class
avg_time_per_class = {cls: np.mean(times) * 1000 for cls, times in class_times.items()}  # in milliseconds
std_time_per_class = {cls: np.std(times) * 1000 for cls, times in class_times.items()}  # std in ms

# Convert to DataFrame
time_df = pd.DataFrame({
    'Class': list(avg_time_per_class.keys()),
    'Avg Inference Time (ms)': list(avg_time_per_class.values()),
    'Std (ms)': list(std_time_per_class.values()),
    'Sample Count': [len(class_times[cls]) for cls in avg_time_per_class.keys()]
})

# Sort by time
time_df = time_df.sort_values('Avg Inference Time (ms)', ascending=True).reset_index(drop=True)

# Print table
print("\nInference Time per Class (sorted by speed):")
print(tabulate(time_df, headers="keys", tablefmt="pretty", floatfmt=(".1f", ".3f", ".3f", ".0f")))

# Save to CSV
time_df.to_csv("/kaggle/working/inference_time_per_class.csv", index=False)
print(f"\n✅ Inference time results saved to '/kaggle/working/inference_time_per_class.csv'")

# Plot
plt.figure(figsize=(12, 8))
plt.barh(time_df['Class'], time_df['Avg Inference Time (ms)'], xerr=time_df['Std (ms)'], color='skyblue', edgecolor='black', alpha=0.8)
plt.xlabel('Average Inference Time (ms)')
plt.title(f'Inference Time per Class - {best_attention_type.upper()} Attention Model')
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.savefig("/kaggle/working/inference_time_per_class.png", dpi=150, bbox_inches='tight')
plt.show()